<a href="https://colab.research.google.com/github/jennif8r/notebookLM/blob/main/doclingAndCohereAndChroma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install docling

In [3]:
pip install whisper langchain chromadb cohere pypdfium2 langchain-community

#Tabela e texto


##Passo 1: Configuração e Inicialização


In [4]:
from time import time
import os
import uuid
import chromadb
from chromadb.config import Settings
from cohere import Client
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
import pandas as pd
from docling.document_converter import DocumentConverter

# Inicializar cliente Cohere
cohere_api_key = "senha-aqui"
cohere_client = Client(api_key=cohere_api_key)

# Configurar cliente ChromaDB
client = chromadb.Client()
collection_dimensions = 768  # Dimensão vetorial


###Passo 2: Função para Extrair Conteúdo e Tabelas


In [5]:
def extract_content_and_tables_from_file(file_path):
    converter = DocumentConverter()

    # Converte o arquivo
    result = converter.convert(file_path)
    document = result.document

    # Extrair conteúdo textual
    text_content = document.export_to_text()

    # Extrair tabelas
    tables = []
    for table_ix, table in enumerate(document.tables):
        table_df = table.export_to_dataframe()
        tables.append({
            'index': table_ix + 1,
            'data': table_df
        })

    return text_content, tables


Teste este código para verificar se o conteúdo e as tabelas estão sendo extraídos corretamente:


In [6]:
file_path = "/content/ATIVIDADE_PRATICA_C1_2024.pdf"
text_content, tables = extract_content_and_tables_from_file(file_path)
print("Conteúdo extraído:\n", text_content)
for table in tables:
    print(f"Tabela {table['index']}:\n", table['data'])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Conteúdo extraído:
 <missing-text>

Fundamentos da Programação Web Atividade Prática

ATIVIDADE PRÁTICA

1.  OBJETIVO

Aplicar os conceitos estudados na disciplina.

2.  PROGRAMAS NECESSÁRIOS

Para desenvolver esta atividade será necessário:

 ❖ HTML

 ❖ CSS

 ❖ JavaScript (livre)

 ❖ Acesso a um provedor de hospedagem gratuito.

 ❖ Um diagramador (IDE VS Code, por exemplo)

3.  ORIENTAÇÕES GERAIS

Desenvolver um site, totalmente funcional, com no mínimo quatro links, conforme orientações, e publicá-lo num servidor de hospedagem gratuito. Procure  deixar  o  menu  visível  em  todas  as  páginas,  evite  o  uso  do VOLTAR do navegador.

Exemplos:

<missing-text>

<missing-text>

Fundamentos da Programação Web Atividade Prática

Ou

<missing-text>

 O tema do site: Portfólio pessoal (CV online).

Requisitos:

 ❖ Um link Sobre mim: que fale sobre você, hobbies etc;

 ❖ Um link sobre sua formação educacional, idiomas;

 ❖ Um link com seu portfólio. Pode colocar links para outros sites que

##Passo 3: Divisão em Chunks
Dividimos o conteúdo extraído em chunks.


In [7]:
def create_chunks_from_text_and_tables(text_content, tables, file_name, chunk_size=500):
    # Usa LangChain para dividir o texto em chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=100
    )
    text_chunks = text_splitter.create_documents([text_content])
    chunk_id = 1

    # Gera chunks de texto
    combined_chunks = []
    for chunk in text_chunks:
        combined_chunks.append({
            'text': chunk.page_content,
            'metadata': {
                'id': chunk_id,  # Gera um ID único
                'length': len(chunk.page_content),
                'type': 'text',
                'file_name': file_name  # Adiciona o nome do arquivo aos metadados
            }
        })
        chunk_id += 1

    # Gera chunks de tabelas
    for table in tables:
        combined_chunks.append({
            'text': table['data'].to_markdown(),
            'metadata': {
                'id': chunk_id,  # Gera um ID único para tabelas
                'length': len(table['data']),
                'type': 'table',
                'index': table['index'],
                'file_name': file_name  # Adiciona o nome do arquivo aos metadados
            }
        })
        chunk_id += 1

    return combined_chunks


# Testando a função
file_name = os.path.basename(file_path)  # Obtém apenas o nome do arquivo
chunks = create_chunks_from_text_and_tables(text_content, tables, file_name)



Teste a criação dos chunks:

In [8]:
print("Chunks gerados:", len(chunks))

for idx, chunk in enumerate(chunks, start=1):  # Enumerar os chunks a partir de 1
    print(f"\n--- Chunk {idx} ---")
    print(f"Texto do chunk:\n{chunk['text']}")
    print(f"Metadados: {chunk['metadata']}")


Chunks gerados: 12

--- Chunk 1 ---
Texto do chunk:
<missing-text>

Fundamentos da Programação Web Atividade Prática

ATIVIDADE PRÁTICA

1.  OBJETIVO

Aplicar os conceitos estudados na disciplina.

2.  PROGRAMAS NECESSÁRIOS

Para desenvolver esta atividade será necessário:

 ❖ HTML

 ❖ CSS

 ❖ JavaScript (livre)

 ❖ Acesso a um provedor de hospedagem gratuito.

 ❖ Um diagramador (IDE VS Code, por exemplo)

3.  ORIENTAÇÕES GERAIS
Metadados: {'id': 1, 'length': 380, 'type': 'text', 'file_name': 'ATIVIDADE_PRATICA_C1_2024.pdf'}

--- Chunk 2 ---
Texto do chunk:
❖ Um diagramador (IDE VS Code, por exemplo)

3.  ORIENTAÇÕES GERAIS

Desenvolver um site, totalmente funcional, com no mínimo quatro links, conforme orientações, e publicá-lo num servidor de hospedagem gratuito. Procure  deixar  o  menu  visível  em  todas  as  páginas,  evite  o  uso  do VOLTAR do navegador.

Exemplos:

<missing-text>

<missing-text>

Fundamentos da Programação Web Atividade Prática

Ou

<missing-text>

 O tema do 

##Passo 4: Gerar Embeddings
Utilizamos o Cohere para gerar embeddings.


In [9]:
def generate_embeddings(text_list):
    response = cohere_client.embed(model="embed-multilingual-v2.0", texts=text_list)
    return response.embeddings


Teste a geração dos embeddings:


In [10]:
texts = [chunk['text'] for chunk in chunks]
embeddings = generate_embeddings(texts)
print("Quantidade de embeddings gerados:", len(embeddings))
for i, embedding in enumerate(embeddings[:3]):  # Mostre os primeiros 3 embeddings
    print(f"Embedding {i + 1}: {embedding[:10]}...")  # Exibir os primeiros 10 valores


Quantidade de embeddings gerados: 12
Embedding 1: [0.50097656, 0.11077881, 0.20092773, 0.25708008, -0.3779297, 0.20288086, 0.4411621, -0.39990234, 0.43066406, 0.18347168]...
Embedding 2: [0.25073242, 0.41381836, -0.13146973, 0.40551758, 0.18371582, 0.0635376, 0.33422852, -0.38134766, 0.1550293, 0.4831543]...
Embedding 3: [0.16003418, 0.3708496, -0.00056123734, 0.07409668, 0.05493164, -0.08178711, 0.3239746, -0.30737305, 0.12683105, 0.04107666]...


Combinando os embedding com os metadados

In [11]:
# Combinar embeddings com seus metadados, incluindo o campo 'content'
def combine_embeddings_with_metadata(embeddings, chunks):
    combined_data = []
    for embedding, chunk in zip(embeddings, chunks):
        combined_data.append({
            'embedding': embedding,  # O embedding gerado
            'metadata': {
                **chunk['metadata'],  # Mantém os metadados originais
                'content': chunk['text']  # Adiciona o conteúdo do chunk
            }
        })
    return combined_data

# Combinar embeddings e metadados
combined_data = combine_embeddings_with_metadata(embeddings, chunks)

# Exibir os dados combinados para validação
print("Quantidade de dados combinados:", len(combined_data))
for idx, data in enumerate(combined_data[:12], start=1):  # Mostrar os primeiros 3 para validação
    print(f"\n--- Dado {idx} ---")
    print(f"Embedding (resumido): {data['embedding'][:10]}...")  # Mostrar apenas os primeiros 10 valores
    print(f"Metadados: {data['metadata']}")


Quantidade de dados combinados: 12

--- Dado 1 ---
Embedding (resumido): [0.50097656, 0.11077881, 0.20092773, 0.25708008, -0.3779297, 0.20288086, 0.4411621, -0.39990234, 0.43066406, 0.18347168]...
Metadados: {'id': 1, 'length': 380, 'type': 'text', 'file_name': 'ATIVIDADE_PRATICA_C1_2024.pdf', 'content': '<missing-text>\n\nFundamentos da Programação Web Atividade Prática\n\nATIVIDADE PRÁTICA\n\n1.  OBJETIVO\n\nAplicar os conceitos estudados na disciplina.\n\n2.  PROGRAMAS NECESSÁRIOS\n\nPara desenvolver esta atividade será necessário:\n\n ❖ HTML\n\n ❖ CSS\n\n ❖ JavaScript (livre)\n\n ❖ Acesso a um provedor de hospedagem gratuito.\n\n ❖ Um diagramador (IDE VS Code, por exemplo)\n\n3.  ORIENTAÇÕES GERAIS'}

--- Dado 2 ---
Embedding (resumido): [0.25073242, 0.41381836, -0.13146973, 0.40551758, 0.18371582, 0.0635376, 0.33422852, -0.38134766, 0.1550293, 0.4831543]...
Metadados: {'id': 2, 'length': 485, 'type': 'text', 'file_name': 'ATIVIDADE_PRATICA_C1_2024.pdf', 'content': '❖ Um diagramado

##Passo 5: Inserir no ChromaDB
Adicionamos os embeddings e metadados ao ChromaDB.

In [12]:
# Criar ou obter a coleção no ChromaDB
collection_name = "novo"
collection = client.get_or_create_collection(
    name=collection_name,
    metadata={"dimensions": collection_dimensions}
)

# Inserir embeddings, metadados e IDs na coleção
for chunk, embedding in zip(chunks, embeddings):
    # Atualizar os metadados para incluir o campo 'content'
    metadata_with_content = {
        **chunk['metadata'],  # Mantém os metadados originais
        'content': chunk['text']  # Adiciona o conteúdo do chunk como metadado
    }

    # Adicionar ao banco vetorial
    collection.add(
        embeddings=[embedding],  # Embedding gerado
        metadatas=[metadata_with_content],  # Metadados atualizados com 'content'
        ids=[str(chunk['metadata']['id'])]  # ID único do chunk
    )

print(f"{len(embeddings)} registros inseridos na coleção '{collection_name}' com sucesso!")


12 registros inseridos na coleção 'novo' com sucesso!


Teste se os dados foram inseridos corretamente:

In [13]:
# Recuperar os dados da coleção para validação
all_data = collection.get(include=["metadatas", "embeddings"])

# Exibir os primeiros registros para validação
print(f"Total de registros na coleção: {len(all_data['metadatas'])}")
for idx, (metadata, embedding) in enumerate(zip(all_data['metadatas'], all_data['embeddings']), start=1):
    print(f"\n--- Registro {idx} ---")
    print(f"Metadados: {metadata}")
    print(f"Embedding (resumido): {embedding[:10]}...")  # Exibe apenas os primeiros 10 valores do embedding


Total de registros na coleção: 12

--- Registro 1 ---
Metadados: {'content': '<missing-text>\n\nFundamentos da Programação Web Atividade Prática\n\nATIVIDADE PRÁTICA\n\n1.  OBJETIVO\n\nAplicar os conceitos estudados na disciplina.\n\n2.  PROGRAMAS NECESSÁRIOS\n\nPara desenvolver esta atividade será necessário:\n\n ❖ HTML\n\n ❖ CSS\n\n ❖ JavaScript (livre)\n\n ❖ Acesso a um provedor de hospedagem gratuito.\n\n ❖ Um diagramador (IDE VS Code, por exemplo)\n\n3.  ORIENTAÇÕES GERAIS', 'file_name': 'ATIVIDADE_PRATICA_C1_2024.pdf', 'id': 1, 'length': 380, 'type': 'text'}
Embedding (resumido): [ 0.50097656  0.11077881  0.20092773  0.25708008 -0.37792969  0.20288086
  0.44116211 -0.39990234  0.43066406  0.18347168]...

--- Registro 2 ---
Metadados: {'content': '❖ Um diagramador (IDE VS Code, por exemplo)\n\n3.  ORIENTAÇÕES GERAIS\n\nDesenvolver um site, totalmente funcional, com no mínimo quatro links, conforme orientações, e publicá-lo num servidor de hospedagem gratuito. Procure  deixar  o  m

##Passo 6: Depurando a Consulta


Verifique se os embeddings estão disponíveis no ChromaDB.


In [14]:
results = collection.query(
    query_embeddings=[embeddings[0]],  # Use o primeiro embedding como exemplo
    n_results=2
)
print("Resultados da consulta:", results)


Resultados da consulta: {'ids': [['1', '2']], 'embeddings': None, 'documents': [[None, None]], 'uris': None, 'data': None, 'metadatas': [[{'content': '<missing-text>\n\nFundamentos da Programação Web Atividade Prática\n\nATIVIDADE PRÁTICA\n\n1.  OBJETIVO\n\nAplicar os conceitos estudados na disciplina.\n\n2.  PROGRAMAS NECESSÁRIOS\n\nPara desenvolver esta atividade será necessário:\n\n ❖ HTML\n\n ❖ CSS\n\n ❖ JavaScript (livre)\n\n ❖ Acesso a um provedor de hospedagem gratuito.\n\n ❖ Um diagramador (IDE VS Code, por exemplo)\n\n3.  ORIENTAÇÕES GERAIS', 'file_name': 'ATIVIDADE_PRATICA_C1_2024.pdf', 'id': 1, 'length': 380, 'type': 'text'}, {'content': '❖ Um diagramador (IDE VS Code, por exemplo)\n\n3.  ORIENTAÇÕES GERAIS\n\nDesenvolver um site, totalmente funcional, com no mínimo quatro links, conforme orientações, e publicá-lo num servidor de hospedagem gratuito. Procure  deixar  o  menu  visível  em  todas  as  páginas,  evite  o  uso  do VOLTAR do navegador.\n\nExemplos:\n\n<missing-te

In [15]:
# Realizar a consulta no ChromaDB
results = collection.query(
    query_embeddings=[embeddings[11]],  # Use o primeiro embedding como exemplo
    n_results=12
)

# Validar e exibir os resultados
if results and 'metadatas' in results and results['metadatas']:
    print("Resultados da consulta:")
    for idx, (metadata, score_list) in enumerate(zip(results['metadatas'], results['distances']), start=1):
        # Se score_list for uma lista, pegue o primeiro elemento
        score = score_list[0] if isinstance(score_list, list) else score_list
        print(f"\n--- Resultado {idx} ---")
        print("Metadados:", metadata)
        print(f"Similaridade: {1 - score:.4f}")  # Similaridade como 1 - distância (se for cosseno)
else:
    print("Nenhum resultado encontrado.")


Resultados da consulta:

--- Resultado 1 ---
Metadados: [{'content': '|    | Páginas   | Atendeu Totalmente  Valor  25  pontos  página                                                                                                                | cada  24  todos   | Atendeu  parcialmente  Valor de 1 a  pontos  cada página                               | Não atendeu  Valor  0  pontos  cada  página   | Nota  cada  página   |\n|---:|:----------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------|:------------------|:---------------------------------------------------------------------------------------|:----------------------------------------------|:---------------------|\n|  0 | Sobre mim | Atendeu                                                                                                                                                      | os                | Atendeu       

In [17]:
collections = client.list_collections()
print([collection.name for collection in collections])


['novo']


In [19]:
print(results['metadatas'])


[[{'content': '|    | Páginas   | Atendeu Totalmente  Valor  25  pontos  página                                                                                                                | cada  24  todos   | Atendeu  parcialmente  Valor de 1 a  pontos  cada página                               | Não atendeu  Valor  0  pontos  cada  página   | Nota  cada  página   |\n|---:|:----------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------|:------------------|:---------------------------------------------------------------------------------------|:----------------------------------------------|:---------------------|\n|  0 | Sobre mim | Atendeu                                                                                                                                                      | os                | Atendeu                                                              

In [30]:
import cohere

def search_and_generate_response(query, collection_name, cohere_api_key, n_results=2):
    """
    Realiza busca no banco vetorial e gera uma resposta usando a LLM do Cohere.

    Args:
        query (str): A pergunta ou consulta do usuário.
        collection_name (str): O nome da coleção no ChromaDB.
        cohere_api_key (str): A chave de API do Cohere.
        n_results (int): Número de resultados mais similares a buscar.

    Returns:
        str: A resposta gerada pela LLM do Cohere.
    """
    # Inicializar o cliente Cohere
    cohere_client = cohere.Client(api_key=cohere_api_key)

    # Obter a coleção no ChromaDB
    collection = client.get_collection(name=collection_name)

    # Gerar embedding para a consulta
    query_embedding = cohere_client.embed(
        model="embed-multilingual-v2.0",
        texts=[query]
    ).embeddings[0]

    # Buscar os resultados mais similares no índice vetorial
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results
    )

    # Validar se há resultados
    if not results or not results['metadatas']:
        return "Nenhum resultado relevante encontrado para a sua consulta."

    # Construir o contexto com base nos resultados encontrados
    context = "\n".join(
        f"Trecho {outer_idx + 1}.{inner_idx + 1}: {item['content']}"
        for outer_idx, metadata_list in enumerate(results['metadatas'])
        for inner_idx, item in enumerate(metadata_list)
    )

    # Criar o prompt para a LLM
    prompt = f"""
    Você é um assistente que responde perguntas, usando a arquitetura RAG. Você vai receber as pesquisas mais relevantes e precisa formular uma resposta para o usuário. Pode haver texto e tabelas nos dados fornecidos.

    {context}

    Pergunta do usuário: {query}

    Responda de forma clara e completa com base nos trechos fornecidos.
    """

    # Gerar a resposta usando a LLM do Cohere
    response = cohere_client.generate(
        model="command-r-plus-08-2024",
        prompt=prompt,
        max_tokens=300,
        temperature=0.7
    )
    return response.generations[0].text.strip()


# Teste da Função
collection_name = "novo"
query = "o que tem que fazer na atividade pratica?"
response = search_and_generate_response(query, collection_name, cohere_api_key)

print("Resposta da IA:\n", response)


Resposta da IA:
 A atividade prática visa aplicar os conceitos estudados na disciplina de Fundamentos da Programação Web, exigindo a criação de um formulário de contato visualmente atraente e funcional. Aqui está o que você precisa fazer:

1. **Criação do Formulário de Contato:**
   - Desenvolva um formulário de contato que inclua campos para nome, email e mensagem. Certifique-se de utilizar apenas HTML e CSS, sem o uso de frameworks como Bootstrap.
   - O HTML será usado para estruturar o formulário e os campos de entrada.
   - O CSS será responsável pelo estilo e layout visual do formulário.

2. **Requisitos Técnicos:**
   - **Linguagens:** Utilize apenas HTML e CSS para criar o formulário. Não é permitido o uso de frameworks como Bootstrap ou ferramentas semelhantes.
   - **Estilização:** Crie uma aparência visual atraente para o formulário usando CSS. Garanta que os estilos sejam aplicados corretamente aos elementos do formulário.
   - **Comentários no Código:** Explique seu código